In [12]:
# Importing Libraries
from collections import defaultdict
from operator import itemgetter
from pathlib import Path
import numpy as np
import pandas as pd
from collections import namedtuple
from tabulate import tabulate
import re 

import torch
import os

from adversarial_debiasing import AdversarialDebiasing
from load_data import load_data, transform_data, Datapoint

from load_vectors import load_pretrained_vectors, load_vectors
import config
import utility_functions

import gensim
import gzip
import pickle

In [6]:
# For autoreloading changes made in other python scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# Loading the word vectors dictionary
# For Wikipedia2Vec - use config.wiki_embedding_data_path and config.wiki_embedding_type
# For Glove - use config.glove_embedding_data_path and config.glove_embedding_type
# For GoogleNews (Word2Vec) - use config.google_embedding_data_path and config.google_embedding_type
word_vectors = load_pretrained_vectors(config.google_embedding_data_path, config.save_dir, config.google_embedding_type)


FileNotFoundError: [Errno 2] No such file or directory: 'data/GoogleNews-vectors-negative300.bin.gz'

In [46]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset[0:10]

[Raw_Datapoint(x1='Athens', x2='Greece', x3='Baghdad', y='Iraq', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bangkok', y='Thailand', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Beijing', y='China', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Berlin', y='Germany', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bern', y='Switzerland', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Cairo', y='Egypt', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Canberra', y='Australia', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Hanoi', y='Vietnam', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Havana', y='Cuba', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Helsinki', y='Finland', task='capital-common-cou

In [48]:
# Transform the data such that it includes the embeddings of the words in consideration
transformed_analogy_dataset, gender_subspace = transform_data(word_vectors, analogy_dataset, use_boluk = False)


# Obtaining the dimensionality of the word embeddings
word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]

# Testing the transformed analogy dataset
assert transformed_analogy_dataset[0].analogy_embeddings.shape[0] == word_embedding_dim * 3
assert transformed_analogy_dataset[0].gt_embedding.shape[0] == word_embedding_dim
assert transformed_analogy_dataset[0].protected.shape[0] == 1

print(transformed_analogy_dataset[0].analogy_embeddings.shape)
print(transformed_analogy_dataset[0].gt_embedding.shape)
print(transformed_analogy_dataset[0].protected.shape)

NameError: name 'word_vectors' is not defined

In [ ]:
# To run the grid-search and obtain the np.dot(w.T, g) values
learning_rate_list = [2 ** -12, 2 ** -6, 2 ** -3]
adversary_loss_weight_list = [1.0, 0.5, 0.1]

# For the saved model checkpoints pertaining to the word embedding type
word_embedding_type = 'GNews'

# Performing the grid search
utility_functions.grid_search(learning_rate_list, adversary_loss_weight_list, word_embedding_dim, gender_subspace, transformed_analogy_dataset, word_embedding_type, 'models')


In [63]:
gender_subspace = torch.zeros(word_embedding_dim)

def load_model(model_path: Path, word_embedding_dim, gender_subspace):
    # with open(str(model_path), "rb") as f:
    #     state_dict = pickle.load(f)
    # device = torch.device('cpu')
    # print(model_path)
    state_dict = torch.load(str(model_path), map_location=torch.device('cpu'))
    model = AdversarialDebiasing(
                    seed = 42,
                    word_embedding_dim = word_embedding_dim,
                    num_epochs = 500,
                    debias = False,
                    gender_subspace = gender_subspace,
                    batch_size = 256,
                    adversary_loss_weight = 0.1,
                    classifier_learning_rate = 2 ** -6,
                    adversary_learning_rate = 2 ** -6
                )
    
    model.W1 = state_dict["W1"]
    model.W2 = state_dict["W2"]
    
    return model

debiased_models = []
non_debiased_models = []


ModelResult = namedtuple('ModelResult', ['best_model', 'last_model', 'embedding_type', 'learning_rate', 'adversary_weight', 'debiased'])

for model_base_path in [Path('models/debiased'), Path('models/non_debiased')]:
    l, debiased = (non_debiased_models, False) if 'non_debiased' in str(model_base_path) else (debiased_models, True)

    for model_path in model_base_path.iterdir():
        if '_last' in str(model_path):
            continue
            
        m = re.search('^([A-Za-z]+)_([\d.]+)_([\d.]+)(_last){0,1}.pckl$', str(model_path.name))
        embeddings = m.group(1)
        learning_rate = m.group(2)
        adversary_weight = m.group(3)
        
        best_model = load_model(model_path, word_embedding_dim, gender_subspace)
        
        last_model_path = model_path.parent / f"{model_path.stem}_last{model_path.suffix}"
        last_model = load_model(last_model_path, word_embedding_dim, gender_subspace)
        
        
        l.append(ModelResult(best_model, last_model, embeddings, learning_rate, adversary_weight, debiased))
        
        

In [64]:
print(debiased_models, len(debiased_models))
print(non_debiased_models, len(non_debiased_models))


[ModelResult(best_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e828b4610>, last_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e8084b3d0>, embedding_type='GNews', learning_rate='0.125', adversary_weight='1.0', debiased=True), ModelResult(best_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e828b4ad0>, last_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e828081d0>, embedding_type='WikipediaVec', learning_rate='0.125', adversary_weight='0.5', debiased=True), ModelResult(best_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e80863850>, last_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e808639d0>, embedding_type='WikipediaVec', learning_rate='0.000244140625', adversary_weight='0.5', debiased=True), ModelResult(best_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e808636d0>, last_model=<adversarial_debiasing.AdversarialDebiasing object at 0x7f5e80867bd0>, embedding_t

In [72]:
learning_rates = list(set(x.learning_rate for x in debiased_models)) 
adversary_weights = list(set(x.adversary_weight for x in debiased_models)) 


box_df = pd.DataFrame([], columns=learning_rates, index=adversary_weights)

for model_result in debiased_models:
    box_df.loc[model_result.adversary_weight, model_result.learning_rate] = np.dot(model_result.last_model.W1.clone().detach().numpy().T, gender_subspace).item()

box_df
# debiased_model_best.W1 = best_state_dict["W1"]
# debiased_model_best.W2 = best_state_dict["W2"]
# 
# debiased_model_last.W1 = last_state_dict["W1"]
# debiased_model_last.W2 = last_state_dict["W2"]
# 
# print("Best : {}".format(np.dot(debiased_model_best.W1.clone().detach().cpu().numpy().T, gender_subspace.T)))
# 
# print("Last : {}".format(np.dot(debiased_model_last.W1.clone().detach().cpu().numpy().T, gender_subspace.T)))

,0.125,0.015625,0.000244140625
0.1,0.0,0.0,0.0
1.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0


In [ ]:
# word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]
# # Training the variant of the model without debiasing
# non_debiased_model = AdversarialDebiasing(
#     word_embedding_dim=word_embedding_dim,
#     num_epochs=500,
#     debias=False,
#     gender_subspace=gender_subspace,
#     batch_size=256,
#     adversary_loss_weight=0.1,
#     classifier_learning_rate = 2 ** -6,
#     adversary_learning_rate = 2 ** -6
# )
# non_debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = non_debiased_model.get_model_weights()
print(np.dot(W1.detach().numpy().T,gender_subspace.T))

In [ ]:
gender_subspace.shape

In [ ]:
# Training the variant of the model with debiasing
# debiased_model = AdversarialDebiasing(
#     word_embedding_dim=word_embedding_dim,
#     num_epochs=500,
#     debias=True,
#     gender_subspace=gender_subspace,
#     batch_size=256,
#     adversary_loss_weight=0.1,
#     classifier_learning_rate = 2 ** -8,
#     adversary_learning_rate = 2 ** -8
# )
debiased_model = AdversarialDebiasing(
    word_embedding_dim=word_embedding_dim,
    num_epochs=500,
    debias=True,
    gender_subspace=gender_subspace,
    batch_size=256,
    adversary_loss_weight=0.1,
    classifier_learning_rate = 2 ** -6,
    adversary_learning_rate = 2 ** -6
)

debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = debiased_model.get_model_weights()
print(np.dot(W1.clone().cpu().detach().numpy().T,gender_subspace.T))

In [ ]:
# Examples to test the models upon
datapoints, test_analogies = [], []
with open(os.path.join('data', 'sexism-traps.txt'), 'r') as f:
    # Reading each line
    for line in f.readlines():
        words = line.split()
        if words[0] == ':':
            continue
        test_analogies.append(words)
        word_embeddings = word_vectors[words]
        word_embeddings = np.reshape(word_embeddings, (1, -1))
        datapoints.append(word_embeddings)
datapoints = np.vstack(datapoints)
print(datapoints.shape)

In [ ]:
# Qualitative evaluation of the non-debiased model

# non_debiased_predictions = non_debiased_model.predict(datapoints)
features = torch.cat([torch.Tensor(x).unsqueeze_(0) for x in datapoints])
x1 = features[:, 0:word_embedding_dim]
x2 = features[:, word_embedding_dim:word_embedding_dim * 2]
x3 = features[:, word_embedding_dim * 2:word_embedding_dim * 3]

non_debiased_predictions = x2 + x3 - x1
non_debiased_predictions = non_debiased_predictions.cpu().detach().numpy()

In [ ]:
non_debiased_most_similar_list = utility_functions.obtain_most_similar(non_debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the non-debiased model
non_debiased_most_similar_list_data_frames = []
for i in range(len(non_debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(non_debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    non_debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Qualitative evaluation of the debiased model
debiased_predictions = debiased_model.predict(datapoints)
debiased_most_similar_list = utility_functions.obtain_most_similar(debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the debiased model
debiased_most_similar_list_data_frames = []
for i in range(len(debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Combining the dataframes pertaining to both the variants of the model
iterables = [['Biased', 'Debiased'], ['Neighbour', 'Similarity']]
index = pd.MultiIndex.from_product(iterables)
overall_data_frames_list = []
for i in range(len(non_debiased_most_similar_list)):
    overall_list = []
    print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    for j in range(1, len(non_debiased_most_similar_list[i])):
        temp_list = []
        temp_list.append(non_debiased_most_similar_list[i][j][0])
        temp_list.append(round(non_debiased_most_similar_list[i][j][1], 3))
        temp_list.append(debiased_most_similar_list[i][j][0])
        temp_list.append(round(debiased_most_similar_list[i][j][1], 3))
        overall_list.append(temp_list)
    temp_df = pd.DataFrame(overall_list, columns = index)
    # print(temp_df.to_string(index = False))
    print(tabulate(temp_df, headers = ['Biased\nNeighbour', 'Biased\nSimilarity', 'Debiased\nNeighbour', 'Debiased\nSimilarity'], tablefmt = 'psql', showindex = False))
    overall_data_frames_list.append(temp_df)